# Ejercicio 2: Modelo Vectorial y TF-IDF

## Objetivo de la práctica

- Comprender el modelo vectorial como base para representar documentos y consultas.
- Calcular la matriz TF-IDF para el corpus `data/01_corpus_turismo_500.txt`

### Nombre: Michael Perugachi            
### Grupo: GR1CC


In [18]:
import pandas as pd
import numpy as np
import string

## Leer el documento 

In [19]:
ruta = "/kaggle/input/01-corpus-turismo-500-txt/01_corpus_turismo_500.txt"   # ajusta si tu ruta es distinta
with open(ruta, "r", encoding="utf-8") as f:
    lineas = f.readlines()
lineas

['Otavalo es conocido por su mercado indígena y su artesanía Perfecto para rafting.\n',
 'La Laguna Quilotoa destaca por su color turquesa\n',
 'Vilcabamba atrae visitantes interesados en longevidad y naturaleza Una experiencia inolvidable.\n',
 'Quito tiene un centro histórico Patrimonio de la Humanidad Perfecto para surf.\n',
 'Ecuador es un país megadiverso, ideal para el turismo ecológico\n',
 'Los turistas disfrutan las fiestas y gastronomía típica local Perfecto para avistamiento de aves.\n',
 'Otavalo es conocido por su mercado indígena y su artesanía Una experiencia inolvidable.\n',
 'El volcán Cotopaxi es un destino de senderismo popular\n',
 'El Parque Nacional Cajas invita a caminatas entre lagunas Ideal para el próximo feriado.\n',
 'Mindo es famoso por el avistamiento de aves Ideal para el próximo feriado.\n',
 'Cuenca deslumbra con su arquitectura colonial y gastronomía Una experiencia inolvidable.\n',
 'Las Islas Galápagos atraen visitantes por su biodiversidad única Ide

## 2. Cargar el corpus
### - Limpiamos las líneas vacías y los saltos de línea.
### - El resultado será una lista donde cada elemento corresponde a un documento.

In [20]:
corpus = [ln.strip() for ln in lineas if ln.strip() != ""]
print(f"Documentos cargados: {len(corpus)}")

Documentos cargados: 500


## 3.Preprocesamiento de datos 
#### - Convertimos todo el texto a minúsculas
#### - Eliminamos signos de puntuación
#### - Tokenizamos palabra por palabra (dividimos por espacios)

In [21]:
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar signos de puntuación
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Dividir el texto en palabras
    tokens = [t for t in text.split() if t != ""]
    return tokens

# Aplicar el preprocesamiento a cada documento del corpus
corpus_tokens = [preprocess_text(doc) for doc in corpus]
corpus_tokens

[['otavalo',
  'es',
  'conocido',
  'por',
  'su',
  'mercado',
  'indígena',
  'y',
  'su',
  'artesanía',
  'perfecto',
  'para',
  'rafting'],
 ['la', 'laguna', 'quilotoa', 'destaca', 'por', 'su', 'color', 'turquesa'],
 ['vilcabamba',
  'atrae',
  'visitantes',
  'interesados',
  'en',
  'longevidad',
  'y',
  'naturaleza',
  'una',
  'experiencia',
  'inolvidable'],
 ['quito',
  'tiene',
  'un',
  'centro',
  'histórico',
  'patrimonio',
  'de',
  'la',
  'humanidad',
  'perfecto',
  'para',
  'surf'],
 ['ecuador',
  'es',
  'un',
  'país',
  'megadiverso',
  'ideal',
  'para',
  'el',
  'turismo',
  'ecológico'],
 ['los',
  'turistas',
  'disfrutan',
  'las',
  'fiestas',
  'y',
  'gastronomía',
  'típica',
  'local',
  'perfecto',
  'para',
  'avistamiento',
  'de',
  'aves'],
 ['otavalo',
  'es',
  'conocido',
  'por',
  'su',
  'mercado',
  'indígena',
  'y',
  'su',
  'artesanía',
  'una',
  'experiencia',
  'inolvidable'],
 ['el',
  'volcán',
  'cotopaxi',
  'es',
  'un',
  

## 4. Construir el diccionario de términos (vocabulario)
### - Se construye un conjunto con todas las palabras únicas del corpus.
### - Luego se convierte en una lista ordenada para tener índices consistentes.

In [22]:
vocab_set = set()
for tokens in corpus_tokens:
    vocab_set.update(tokens)

vocab = sorted(vocab_set)   # Lista de términos únicos ordenados alfabéticamente
V = len(vocab)              # Tamaño del vocabulario
N = len(corpus_tokens)      # Número de documentos

print(f"Vocabulario total: {V} términos")

# Crear un diccionario término → índice de columna
term_to_idx = {term: i for i, term in enumerate(vocab)}

Vocabulario total: 118 términos


## 5. Cálculo de TF (Term Frequency) 
### - TF mide la frecuencia relativa de cada término en un documento.

In [37]:
TF = np.zeros((N, V), dtype=float)  # matriz de ceros tamaño (N x V)

for doc_i, tokens in enumerate(corpus_tokens):
    if len(tokens) == 0:
        continue
    # Contar ocurrencias de cada término en el documento
    counts = {}
    for t in tokens:
        counts[t] = counts.get(t, 0) + 1

    total_terms = len(tokens)
    # Calcular frecuencia relativa y llenar la matriz TF
    for term, cnt in counts.items():
        j = term_to_idx[term]
        TF[doc_i, j] = cnt / total_terms

## 6. Cálculo de IDF (Inverse Document Frequency)
### - IDF mide qué tan importante es un término dentro del corpus.
### - Fórmula: IDF(term) = log(N / df(term))
### - Donde df(term) = número de documentos que contienen el término.

In [38]:
df = np.zeros(V, dtype=float)  # vector para almacenar df de cada término

for j, term in enumerate(vocab):
    cnt_docs = 0
    for tokens in corpus_tokens:
        if term in tokens:
            cnt_docs += 1
    df[j] = cnt_docs

# Calcular IDF evitando división por cero
idf = np.zeros(V, dtype=float)
for j in range(V):
    if df[j] > 0:
        idf[j] = np.log(N / df[j])  # log natural
    else:
        idf[j] = 0.0

## 7. Cálculo de la matriz TF-IDF
### - Se multiplica cada valor de TF por su correspondiente valor IDF.
### - Fórmula: TF-IDF = TF * IDF

In [39]:

TFIDF = TF * idf[np.newaxis, :]  # Broadcasting: (N x V) * (V,) → (N x V)

# Convertir a DataFrame para visualización más legible
df_tfidf = pd.DataFrame(TFIDF, columns=vocab)

print ("Dimension de Matriz TF-IDF", df_tfidf.shape)

Dimension de Matriz TF-IDF (500, 118)


## 8. Resultados
### - Mostramos las dimensiones y una muestra de la matriz TF-IDF.

In [40]:
print("\nMatriz TF-IDF calculada exitosamente.")
print(f"Dimensiones: {df_tfidf.shape} (Documentos x Términos)")

# Mostrar primeras filas y primeras columnas para referencia
display(df_tfidf.iloc[:5, :20])


Matriz TF-IDF calculada exitosamente.
Dimensiones: (500, 118) (Documentos x Términos)


,2000,a,agua,amazonía,arquitectura,artesanía,atrae,atraen,auténtico,aventura,aves,avistamiento,baños,biodiversidad,cajas,caminatas,canopy,centro,colonial,color
0,0.0,0.0,0.0,0.0,0.0,0.209085,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.343609
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.265343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.26824,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000
